In [1]:
%cd ../..

C:\Users\qaz89\Documents\Git Repos


In [2]:
!cd

C:\Users\qaz89\Documents\Git Repos


In [3]:
import os
import sys
print(sys.version)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]


In [4]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torchvision.transforms as transforms
import torchvision

import pdb
import math
import matplotlib.pyplot as plt
import glob
import PIL
import json